In [1]:
import numpy as np

dem_file = '/home/jens/daten/geometries/usgs_earth_explorer/merged_raster_100m.tif'

T_selected = 50 # at 50 m depth
sealing_selected = 'sealing_100'  # 100 m radiu
#temp = 'T_50'

dataset = 'RLP'
#dataset = 'BW'

logarithmic = False

if dataset == 'BW':
    intercept = 14.43
    beta_elev = -.005
    beta_seal = 0.

    land_use_path = None
    grid_sealing_area = None

    regions_main = [
        'odenwald',
        'black_forest',
        'swabian_alb',
        'voralb',
        'keuper_uplands',
        'alpine_foothills',
        'gaeu',
        'upper_rhine_graben'
    ]
    
    regions_path = '/home/jens/daten/geometries/geological_regions/BW/auswertungsregionen'    
    state_boundary_path = "/home/jens/daten/BW_temperatureLogs_shallow/geometries/Verw-Grenzen_ALKIS/Verwaltungsgrenzen_NOrA_Dez_2022_ALKIS-Shape/v_al_land.shp"

    vmin, vmax, vmin_res, vmax_res = 6, 15, -5, 5
    gridx = np.arange(385000, 615000, 1000)
    gridy = np.arange(5265000, 5525000, 1000)

elif dataset == 'RLP':
    land_use_path = "/home/jens/daten/parameter/lbm-de2021.utm32s.shape/lbm-de2021/land/rp/lbm-de2021.shp" # EPSG:25832
    state_boundary_path = "/home/jens/daten/RLP_temperatureLogs_shallow/geometries/RLP_boundary/landesgrenze_rlp.shp"
    intercept = 12.662277853596251
    beta_elev = -0.008789
    beta_seal = 0.012684

    regions_main = []
    if True:
        regions_main = [
            'rhine_graben_slab',
            'rhine_graben_zwischenscholle',
            'mainz_basin_tertiary',
            'quaternary_north_west',
            'quaternary_middle_rhine',
            'tertiary_quaternary_rhine_main',
            'palatinate_buntsandstein',
            'palatinate_muschelkalk',
            'triassic_north_west',
            'tertiary_north_west',    
            'palatinate_permokarbon',
            'perm_nahe_prims_basin',
            'palatinate_slate_mountain_range_south',
            'palatinate_slate_mountain_range_north',
            'lahn_dill',
            'westerwald_tertiary',
            'limestone_depression_buntsandstein_vulkanite',
            'kaenozoic_vulkanite'   
        ]
    
    regions_path = '/home/jens/daten/geometries/geological_regions/RLP/hydrogeology'

    # Total RLP
    #vmin, vmax, vmin_res, vmax_res = 6, 15, -5, 5
    #grid_sealing_area = 1000*1000    
    #gridx = np.arange(290000, 475000, 1000)
    #gridy = np.arange(5420000, 5650000, 1000)
        
    # Ramstein
    vmin, vmax, vmin_res, vmax_res = 8, 12, -1, 1
    grid_sealing_area = 100*100
    gridx = np.arange(392100, 404800, 100)
    gridy = np.arange(5469600, 5482300, 100)


def trend_global(_elev, _seal):
    return intercept + beta_elev*_elev + beta_seal*_seal

<h1> Import </h1>

<h2> Modules </h2>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import gstools as gs
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
import geopandas as gpd
from pyproj import Transformer

from sqlalchemy import create_engine

from bokeh.plotting import figure, show, output_file
from bokeh.models import LinearColorMapper, ColorBar, HoverTool, ColumnDataSource, Range1d, LinearAxis, GeoJSONDataSource, Label
from bokeh.transform import linear_cmap, cumsum
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.resources import INLINE
from bokeh.palettes import Reds256, Oranges256, Blues256, Greens256, RdBu, YlGn, RdYlGn, linear_palette, YlOrBr, Viridis256

from shapely.geometry import Point
from matplotlib.colors import LinearSegmentedColormap

# output_notebook()

<h2> DEM </h2>

In [ ]:
src = rasterio.open(dem_file)

xx, yy = np.meshgrid(gridx, gridy)

transformer = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)
_xx, _yy =  transformer.transform(xx, yy)

gdf_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(_xx.flatten(), _yy.flatten()) )
gdf_points.crs="EPSG:4326"
gdf_points = gdf_points.to_crs(src.crs)

grid_elevation_values = np.zeros(_xx.size)

print(f'{xx.size} Points')
for ndx, point in enumerate(gdf_points.geometry):
    if ndx%1000 == 0:
        print(f'\tPoint {ndx}')
    x, y = point.x, point.y
    row, col = src.index(x, y)
    #elevation = np.flipud(src.read(1))[row, col]
    #elevation = src.read(1).T[row, col]
    elevation = src.read(1)[row, col] # 200
    grid_elevation_values[ndx] = elevation

######
dst_crs = "EPSG:32632"

#grid_elevation_values = grid_elevation_values.T
#grid_elevation_values = np.flipud(grid_elevation_values)

# Berechne Transformation und neue Dimensionen
transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)

# Initialisiere ein Array für die transformierten Daten
transformed_dem = np.empty((height, width), dtype=src.meta['dtype'])

# Transformiere das Raster
reproject(
    source=src.read(1),
    destination=transformed_dem,
    src_transform=src.transform,
    src_crs=src.crs,
    dst_transform=transform,
    dst_crs=dst_crs,
    resampling=Resampling.nearest
)

dem_extent = [transform[2], transform[2] + width * transform[0], transform[5] + height * transform[4], transform[5]]

grid_elevation_values = grid_elevation_values.reshape(xx.shape).T

#grid_elevation_values = np.ones_like(grid_elevation_values) * 250

print('Finished')

16129 Points
	Point 0
	Point 1000
	Point 2000
	Point 3000
	Point 4000


In [ ]:
def calculate_sealing_on_grid(gridx, gridy, gdf_versiegelung, radius, value_column='SIE_AKT'):
    """
    Berechnet den gewichteten Versiegelungsgrad für jeden Punkt eines regulären Grids.

    Parameter:
    -----------
    gridx, gridy : 1D numpy arrays
        Koordinaten des Rasters (in EPSG:32632).
    gdf_versiegelung : GeoDataFrame
        Polygone mit Versiegelungswerten (z. B. Bodenversiegelung), CRS = EPSG:32632.
    radius : float
        Suchradius in Metern.
    value_column : str
        Name der Spalte im Polygon-Layer mit Versiegelungsgrad (0–100 oder 0–1).

    Rückgabe:
    ----------
    grid_sealing_values : 2D numpy array (shape = (len(gridy), len(gridx)))
        Gitter mit gewichteten Versiegelungswerten.
    """

    # Meshgrid für alle Punkte
    xx, yy = np.meshgrid(gridx, gridy)
    flat_points = np.column_stack([xx.flatten(), yy.flatten()])

    print(f"{flat_points.shape[0]} grid points → buffering radius {radius:.1f} m")

    # GeoDataFrame aus Gitterpunkten
    gdf_points = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy(flat_points[:,0], flat_points[:,1]),
        crs="EPSG:32632"
    )

    # Buffer um jeden Gitterpunkt
    gdf_buffer = gdf_points.copy()
    gdf_buffer["geometry"] = gdf_buffer.buffer(radius)
    gdf_buffer["grid_index"] = gdf_buffer.index

    # Overlay mit Versiegelungspolygonen
    intersection = gpd.overlay(gdf_buffer, gdf_versiegelung, how="intersection")
    print(f"Overlay intersections: {len(intersection)}")

    if intersection.empty:
        print("No intersections – all sealing values set to 0")
        return np.zeros(xx.shape)

    # Flächenberechnung
    intersection["area"] = intersection.geometry.area

    # Gewichtete Versiegelung pro Rasterzelle
    def weighted_sealing(gr):
        total_area = gr["area"].sum()
        if total_area == 0:
            return 0.0
        return (gr[value_column] * gr["area"]).sum() / total_area

    sealing_series = (
    intersection.drop(columns=["grid_index"])
    .groupby(intersection["grid_index"])
    .apply(weighted_sealing))

    # Ergebnisse wieder in Gitterform bringen
    grid_sealing_values = sealing_series.reindex(
        range(flat_points.shape[0]), fill_value=np.nan
    ).to_numpy().reshape(xx.shape)

    print("Finished computing sealing grid.")
    return grid_sealing_values


if dataset == 'RLP':
    grid_sealing_radius = np.sqrt(grid_sealing_area / 3.14150)
    print('grid_sealing_radius: ', grid_sealing_radius)

    radius= grid_sealing_radius#np.sqrt(area_selected*area_selected / np.pi)
    
    # Lade Versiegelungsdaten und projiziere richtig
    gdf_versiegelung = gpd.read_file(land_use_path)
    gdf_versiegelung = gdf_versiegelung.to_crs("EPSG:32632")
    
    grid_sealing_values = calculate_sealing_on_grid(gridx, gridy, gdf_versiegelung, radius, value_column='SIE_AKT')
    
    grid_sealing_values = np.nan_to_num(grid_sealing_values, nan=0.0)
    #grid_sealing_values = np.zeros_like(grid_sealing_values)
    
    grid_sealing_values = grid_sealing_values.reshape(xx.shape).T


<h2> Borehole data </h2>

In [ ]:
if dataset == 'BW':
    user, password ='postgres', 'postgres'
    schema, host = 'DB_BW_temperature_shallow', '127.0.0.1'
    connection_string='postgresql://{}:{}@{}:5432/{}'.format(user, password, host, schema)
    
    engine = create_engine(connection_string)
    
    df = pd.read_sql_query("SELECT * FROM temperatures_approximation;", engine)
    df_bh_evaluation = pd.read_sql_query("SELECT * FROM borehole_evaluation;", engine)
    df_infos = pd.read_sql_query("SELECT * FROM log_infos;", engine)
    
    df = pd.merge(df, df_bh_evaluation[
                  ['borehole_id', 'date', 'drillingInduced', 'deactivated', 'quality']], 
                  on='borehole_id', how='inner')
    
    
    df = df.merge(df_infos[['borehole_id', 'utm_easting', 'utm_northing', 
                                'elevation', 'sample_id', 'land_use', 'N112', 'N120', 'N211', 'N311']], on='borehole_id', how='left')
    
    df = df[df['sample_id'] == 'a']
    
    df = df.drop_duplicates()
    df[sealing_selected] = 0

elif dataset == 'RLP': # for sealing at wells
    df = pd.read_csv('df4statstools.csv')


<h2> Filter </h2>

In [ ]:
# filter
print(f'Wells: {df.shape[0]}')


df_filt = df[~df['borehole_id'].astype(str).str.startswith('GT')]
print(f'After removed deppe wells from GeotIS: {df_filt.shape[0]}')

df_filt = df_filt[df_filt['drillingInduced'] == False]
print(f'After removed wells impacted by drilling: {df_filt.shape[0]}')
df_filt = df_filt[df_filt['deactivated'] == False]
print(f'After removed deactivated wells: {df_filt.shape[0]}')

df_filt = df_filt.dropna(subset=[f"T_{T_selected}"])

print(f'And at depth {T_selected}: {df_filt.shape[0]}')


if dataset == 'RLP':
    df_filt = df_filt[df_filt["quality"] > -1 ]
    print(f'After removed quality < 0: {df_filt.shape[0]}')


#print(df_cleaned.shape[0], " boreholes")

# for variogram
factor = 2.5 # um ausreisser im variogram zu entfernen

q1, q3 = np.percentile(df_filt[f"T_{T_selected}"].to_numpy(), [25, 75])
iqr = q3 - q1
lower_bound = q1 - factor * iqr
upper_bound = q3 + factor * iqr


df_filt_cleaned = df_filt[(df_filt[f"T_{T_selected}"]>lower_bound)&(df_filt[f"T_{T_selected}"]<upper_bound) ]

print(df_filt_cleaned.shape[0], f" boreholes after cleaning (factor {factor}) with bounds: {lower_bound} {upper_bound}")


In [ ]:
regions_main = []

<h2> Plot funktions </h2>

In [ ]:
def plot_variogram(title, fit_model, bin_center, gamma, bin_counts, para, r2):
    bin_center = np.insert(bin_center, 0, para['nugget']) # to get position zero in plot 
    source = ColumnDataSource(data={'lags': bin_center/1000, 
                                    'semi_variances': gamma, 
                                    'model_values': fit_model.variogram(bin_center), 
                                    'bin_counts': bin_counts})
    
    semi_variances_max = max(max(gamma), max(fit_model.variogram(bin_center)))*1.1
    
    p = figure(title='',#'range {:.0f}, sill {:.1f}, nugget {:.1f}, Pseudo-R² {:.3f}'.format(
        #para['len_scale'], para['var'], para['nugget'], r2), 
                           x_axis_label='Lag-distance [km]', y_axis_label='Semivariance [K²]', 
                           width=500, height=300, 
                           y_range=Range1d(start=0, end=semi_variances_max))
    
    p.scatter('lags', 'semi_variances', size=8, color='blue', #legend_label='Experimental', 
                     source=source)
    p.line('lags', 'model_values', line_width=2, color='red', #legend_label='Spherical', 
                   source=source)
    #
    p.extra_y_ranges['bin_count_range'] =  Range1d(start=0, end=max(bin_counts)*1.1)
    
    ax2 = LinearAxis(axis_label='N', y_range_name='bin_count_range')
    p.add_layout(ax2, 'right')

    if "var" in para:
        text = Label(x=260, y=35,#  38, 
                     x_units='screen', y_units='screen',
                     #text='\n Range: {:.0f} \n Sill: {:.1f} \n Pseudo-R²: {:.3f} \n'.format(
                     #    para['len_scale'], para['var'], r2),
                     text='\n Range: {:.0f} \n Sill: {:.1f} \n Nugget: {:.1f} \n Pseudo-R²: {:.3f} \n'.format(
                         para['len_scale'], para['var'], para['nugget'], r2),
                     text_font_size='8pt',
                     text_baseline="middle", text_align="left",
                     background_fill_color='white', background_fill_alpha=1)
    else: # double variogram
        if para['var2'] == 0:
            text = Label(x=260, y=35,#  38, 
                     x_units='screen', y_units='screen',
                     #text='\n Range: {:.0f} \n Sill: {:.1f} \n Pseudo-R²: {:.3f} \n'.format(
                     #    para['len_scale'], para['var'], r2),
                    text='\n Range: {:.0f} \n    Sill: {:.1f} \n Nugget: {:.1f} \n\n Pseudo-R²: {:.3f} \n'.format(
                     para['len_scale1'], para['var1'], para['nugget'], r2),
                    text_font_size='8pt',
                    text_baseline="middle", text_align="left",
                    background_fill_color='white', background_fill_alpha=1) 
        else:
            text = Label(x=260, y=70,#  38, 
                         x_units='screen', y_units='screen',
                         #text='\n Range: {:.0f} \n Sill: {:.1f} \n Pseudo-R²: {:.3f} \n'.format(
                         #    para['len_scale'], para['var'], r2),
                        text='\n Near-Field\n    Range: {:.0f} \n    Sill: {:.1f} \n Far-Field\n    Range: {:.0f} \n    Sill: {:.1f} \n Nugget: {:.1f} \n\n Pseudo-R²: {:.3f} \n'.format(
                         para['len_scale1'], para['var1'], para['len_scale2'], para['var2'], para['nugget'], r2),
                        text_font_size='8pt',
                        text_baseline="middle", text_align="left",
                        background_fill_color='white', background_fill_alpha=1)    
    p.add_layout(text)
    
    p.vbar(x='lags', top='bin_counts', width=.9, 
                       color='black', alpha=.5, 
           y_range_name='bin_count_range', source=source)

    show(p)

In [ ]:
def plot_temp(srf, vmin=6, vmax=16):
    fig, axes = plt.subplots()#1, 1, figsize=(20, 5))
        
    ax = axes
    im = ax.imshow(srf.T, origin="lower", cmap='coolwarm', vmin=vmin, vmax=vmax, 
                   extent=(gridx.min(), gridx.max(), gridy.min(), gridy.max()))
    
    cbar = plt.colorbar(im, ax=ax, #orientation="horizontal", 
                        #ticks=boundaries
                       )
    cbar.set_label("Temperature (°C)")
    
    ax.scatter(bhs_x, bhs_y, color='k', zorder=10, marker='+', label='Conditions')
    ax.set_xlim([gridx.min(), gridx.max()])
    ax.set_ylim([gridy.min(), gridy.max()])

def plot_std(var):
    fig, axes = plt.subplots()#1, 1, figsize=(20, 5))
        
    ax = axes
    std = np.sqrt(var.T)
    
    boundaries = np.arange(0, np.ceil(std.max()),# + 0.5, 
                               0.5)
        
    # Normierung: sorgt für harte Übergänge
    norm = mcolors.BoundaryNorm(boundaries=boundaries, ncolors=plt.cm.viridis.N)
    
    im = ax.imshow(std, origin="lower", cmap='viridis', extent=(gridx.min(), gridx.max(), gridy.min(), gridy.max()),
             #vmin=0, vmax=3, 
              norm=norm)
    
    cbar = plt.colorbar(im, ax=ax, #orientation="horizontal", 
                        ticks=boundaries)
    cbar.set_label("Kriging Standardabweichung (σ)")
    
    ax.scatter(bhs_x, bhs_y, color='k', zorder=10, marker='+', label='Conditions')
    ax.set_xlim([gridx.min(), gridx.max()])
    ax.set_ylim([gridy.min(), gridy.max()])

def plot_dem(xx, yy, grid_elevation_values):

    fig, axes = plt.subplots()#1, 1, figsize=(20, 5))
    
    ax = axes
    c = ax.pcolormesh(xx, yy, grid_elevation_values.T, cmap="terrain", shading="auto")
    fig.colorbar(c, ax=ax, label="Elevation [m]")
    ax.scatter(bhs_x, bhs_y, color='k', zorder=10, marker='+', label='Conditions')
    ax.set_xlim([gridx.min(), gridx.max()])
    ax.set_ylim([gridy.min(), gridy.max()])

In [ ]:
def load_dem(path, x_offset, y_offset):
    with rasterio.open(path) as src:
        dem = src.read(1)  # first band
        dem = np.where(dem == src.nodata, np.nan, dem)  # mask nodata values
        bounds = src.bounds
        
        # convert to km relative to offset
        x_min = (bounds.left - x_offset) / 1000
        x_max = (bounds.right - x_offset) / 1000
        y_min = (bounds.bottom - y_offset) / 1000
        y_max = (bounds.top - y_offset) / 1000
        
        dw = x_max - x_min
        dh = y_max - y_min
        
    return dem, x_min, y_min, dw, dh



In [ ]:
aspect_ratio = (gridy.max() - gridy.min()) /(gridx.max()-gridx.min())

def do_T_plot(cbar_title, _srf, x_min, x_max, y_min, y_max, x_range, y_range, tooltips, 
              source, temp_difference=False, logarithmic=False):
    p = figure(
        #title=title,
        x_range=x_range,
        y_range=y_range,
        width=500,
        height=int(500 * aspect_ratio),
        tooltips=tooltips,
        match_aspect=True
    )

    if temp_difference: # residuals
        if vmax-vmin < 3:
            color_mapper_field = LinearColorMapper(palette=RdBu[(vmax-vmin)*4], low=vmin_res, high=vmax_res)
        else:
            color_mapper_field = LinearColorMapper(palette=RdBu[(vmax-vmin)], low=vmin_res, high=vmax_res)
        color_bar = ColorBar(
            color_mapper=color_mapper_field, 
            location=(0, 0), 
            title=cbar_title,
            orientation='horizontal'
        )
    else:
        if vmax-vmin < 5:
            color_mapper_field = LinearColorMapper(palette=RdBu[(vmax-vmin)*2], low=vmin, high=vmax)
        else:
            color_mapper_field = LinearColorMapper(palette=RdBu[(vmax-vmin)], low=vmin, high=vmax)
        # ColorBar mit Titel "Temperatur [°C]" (fett) und Ticks fett
        color_bar = ColorBar(
            color_mapper=color_mapper_field, 
            location=(0, 0), 
            title=cbar_title,
            orientation='horizontal'
        )
    
    p.xaxis.axis_label = "UTM Easting [km]"
    p.yaxis.axis_label = "UTM Northing [km]"
    p.xaxis.axis_label_text_font_style = "bold"
    p.yaxis.axis_label_text_font_style = "bold"
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.major_label_text_font_style = "bold"
    p.title.text_font_size = "10pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    p.yaxis.axis_label_text_font_size = "12pt"

    field = np.exp(_srf.T) if logarithmic else _srf.T
    
    p.image(
        image=[field],
        #x=0,  # entspricht (x_min - x_offset)/1000 = 0 km
        #y=0,  # entspricht 0 km
        x=x_min/1000, y=y_min/1000, # !!
        dw=(x_max - x_min) / 1000,
        dh=(y_max - y_min) / 1000,
        color_mapper=color_mapper_field,
        level="image"
    )

    color_bar.major_label_text_font_size = "12pt"
    color_bar.major_label_text_font_style = "bold"
    color_bar.title_text_font_size = "12pt"
    color_bar.title_text_font_style = "bold"
    p.add_layout(color_bar, 'below')
    
    p.scatter(
        'x',
        'y',
        size=10,
        color='black',
        marker='cross',
        source=source
    )
     
    p.xaxis.major_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label = "UTM Easting [km]"
    p.yaxis.axis_label = "UTM Northing [km]"
    p.xaxis.axis_label_text_font_style = "bold"
    p.yaxis.axis_label_text_font_style = "bold"
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.major_label_text_font_style = "bold"
    p.title.text_font_size = "10pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    p.yaxis.axis_label_text_font_size = "12pt"

    return p

In [ ]:
def do_bplot(title, _dem_file, dem_lim, grids, 
             srf, var, df, df_cleaned, bhs_x, bhs_y, bhs_val, grid_drift=None, logarithmic=False):
    #
    p = [[None] * 3 for _ in range(2)] 
    
    x_min = grids[0].min() # df[df['T_0'].notna()]['utm_easting'].to_numpy().min() #- 40000
    x_max = grids[0].max() # df[df['T_0'].notna()]['utm_easting'].to_numpy().max() #+ 15000
    y_min = grids[1].min() # df[df['T_0'].notna()]['utm_northing'].to_numpy().min() #- 20000
    y_max = grids[1].max() # df[df['T_0'].notna()]['utm_northing'].to_numpy().max() #+ 30000
    
    # Offsets: linke untere Ecke (in Meter) -> entspricht 0 km, 0 km
    x_offset = 0.#x_min # !!
    y_offset = 0.#y_min # !!
    
    # Neue Grenzen in km
    #x_range_km = (0, (x_max - x_min) / 1000)
    #y_range_km = (0, (y_max - y_min) / 1000)
    x_range_km = (x_min / 1000, x_max / 1000) # !!
    y_range_km = (y_min / 1000, y_max / 1000) # !!


    # Für alle Plots dieselben Achsen (in km)
    x_range = x_range_km
    y_range = y_range_km
    
    
    color_mapper_error = LinearColorMapper(palette=linear_palette(YlOrBr[4][::-1], 4), low=0, high=2) 
    
    data = dict(
        x = ((bhs_x- x_offset) / 1000), 
        y = ((bhs_y - y_offset) / 1000), 
        z = bhs_val,
        bh_id = df_cleaned['borehole_id'].to_numpy(),
        quality = df_cleaned['quality'].to_numpy(),
        drillingInduced = df_cleaned['drillingInduced'].to_numpy()
    )
    source = ColumnDataSource(data)
    
    tooltips = [("ID:", "@bh_id"),
                ("T [°C]:", "@z"),
                ("Quality:", "@quality (@drillingInduced)")]
    
    # Temperaturfeld ----
    p[0][0] = do_T_plot("Temperature [°C]", srf, x_min, x_max, y_min, y_max, x_range, y_range, tooltips, source, logarithmic=logarithmic)
    
    if grid_drift is not None:
        p[1][0] = do_T_plot("Drift model [°C]", 
                            grid_drift, x_min, x_max, y_min, y_max, p[0][0].x_range, p[0][0].y_range, 
                            tooltips, source, logarithmic=logarithmic)
        p[1][1] = do_T_plot("Residuals [K]", 
                            srf-grid_drift, x_min, x_max, y_min, y_max, p[0][0].x_range, 
                            p[0][0].y_range, tooltips, source,
                            temp_difference=True, logarithmic=logarithmic)
        
    # ---- Kriging Error ----
    p[0][1] = figure(
        title=title,#"{}, Kriging error".format(title[temp]),
        x_range=p[0][0].x_range,
        y_range=p[0][0].y_range,
        width=500,
        height=int(500 * aspect_ratio),
        match_aspect=True
    )
    
    p[0][1].xaxis.axis_label = "UTM Easting [km]"
    p[0][1].yaxis.axis_label = "UTM Northing [km]"
    p[0][1].xaxis.axis_label_text_font_style = "bold"
    p[0][1].yaxis.axis_label_text_font_style = "bold"
    p[0][1].xaxis.major_label_text_font_style = "bold"
    p[0][1].yaxis.major_label_text_font_style = "bold"
    p[0][1].title.text_font_size = "10pt"
    p[0][1].xaxis.axis_label_text_font_size = "12pt"
    p[0][1].yaxis.axis_label_text_font_size = "12pt"

    p[0][1].image(
        image=[np.sqrt(var.T)],  # Standardabweichung
        #x=0,
        #y=0,
        x=x_min/1000, y=y_min/1000, # !!
        dw=(x_max - x_min) / 1000,
        dh=(y_max - y_min) / 1000,
        color_mapper=color_mapper_error

   )
    
    color_bar_error = ColorBar(
        color_mapper=color_mapper_error, 
        location=(0, 0), 
        title="Kriging standard deviation [K]", 
        orientation='horizontal'
    )
    color_bar_error.major_label_text_font_size = "12pt"
    color_bar_error.major_label_text_font_style = "bold"
    color_bar_error.title_text_font_size = "12pt"
    color_bar_error.title_text_font_style = "bold"
    p[0][1].add_layout(color_bar_error, 'below')
    #p[0][1].patches(xs, ys, fill_alpha=0, line_width=3, line_color="green")

    scattersize = 3
    p[0][1].scatter(
        ((df_cleaned['utm_easting'].to_numpy() - x_offset) / 1000), 
        ((df_cleaned['utm_northing'].to_numpy() - y_offset) / 1000), 
        size=scattersize, color='green', marker='circle'
    )

    p[0][1].xaxis.major_label_text_font_size = "12pt"
    p[0][1].yaxis.major_label_text_font_size = "12pt"


    ###########
    # DEM

    dem_array, dem_x, dem_y, dem_dw, dem_dh = load_dem(_dem_file, x_offset, y_offset)

    color_mapper_dem = LinearColorMapper(palette=RdYlGn[int(dem_lim[1]-dem_lim[0]) / 100], 
                                         low=dem_lim[0], high=dem_lim[1])

    p[0][2] = figure(
        x_range=p[0][0].x_range,
        y_range=p[0][0].y_range,
        width=500,
        height=int(500 * aspect_ratio),
        match_aspect=True
    )

    p[0][2].image(
        image=[np.flipud(dem_array)],  # rasterio reads top->bottom, flip for correct orientation
        #x=0,
        #y=0,
        #dw=(x_max - x_min) / 1000,
        #dh=(y_max - y_min) / 1000,
        x=dem_x,
        y=dem_y,
        dw=dem_dw,
        dh=dem_dh,
        color_mapper=color_mapper_dem
    )

    color_bar_dem = ColorBar(
        color_mapper=color_mapper_dem,
        location=(0, 0),
        title="Land surface elevation [m]",
        orientation="horizontal"
    )

    color_bar_dem.major_label_text_font_size = "12pt"
    color_bar_dem.major_label_text_font_style = "bold"
    color_bar_dem.title_text_font_size = "12pt"
    color_bar_dem.title_text_font_style = "bold"
    p[0][2].add_layout(color_bar_dem, 'below')

    p[0][2].scatter(
        'x',
        'y',
        size=10,
        color='black',
        marker='cross',
        source=source
    )
     
    p[0][2].xaxis.major_label_text_font_size = "12pt"
    p[0][2].yaxis.major_label_text_font_size = "12pt"

    # Achsenbeschriftungen in km (fett) und Ticks fett
    p[0][2].xaxis.axis_label = "UTM Easting [km]"
    p[0][2].yaxis.axis_label = "UTM Northing [km]"
    p[0][2].xaxis.axis_label_text_font_style = "bold"
    p[0][2].yaxis.axis_label_text_font_style = "bold"
    p[0][2].xaxis.major_label_text_font_style = "bold"
    p[0][2].yaxis.major_label_text_font_style = "bold"
    p[0][2].title.text_font_size = "10pt"
    p[0][2].xaxis.axis_label_text_font_size = "12pt"
    p[0][2].yaxis.axis_label_text_font_size = "12pt"
    #p[0][2].patches(xs, ys, fill_alpha=0, line_width=3, line_color="orange")

    ####
    if False:
        shp_path = "/home/jens/daten/BW_temperatureLogs_shallow/geometries/Verw-Grenzen_ALKIS/Verwaltungsgrenzen_NOrA_Dez_2022_ALKIS-Shape/v_al_land.shp"
        gdf = gpd.read_file(shp_path)
    
        # Sicherstellen, dass das CRS stimmt (EPSG:25832 - ETRS89 / UTM zone 32N)
        #if gdf.crs != "EPSG:25832":
        #    gdf = gdf.to_crs("EPSG:25832")
        
        # --- Shapefile-Geometrien extrahieren und umrechnen (auf km und mit Offset) ---
        xs = []
        ys = []
        for geom in gdf.geometry:
            if geom.geom_type == 'Polygon':
                x, y = geom.exterior.coords.xy
                xs.append([(xi - x_offset) / 1000 for xi in x])
                ys.append([(yi - y_offset) / 1000 for yi in y])
            elif geom.geom_type == 'MultiPolygon':
                for poly in geom.geoms:
                    x, y = poly.exterior.coords.xy
                    xs.append([(xi - x_offset) / 1000 for xi in x])
                    ys.append([(yi - y_offset) / 1000 for yi in y])

        line_color='green'
    
        p[0][0].patches(xs, ys, fill_alpha=0, line_width=3, line_color=line_color)
        p[0][1].patches(xs, ys, fill_alpha=0, line_width=3, line_color=line_color)
        if grid_drift is not None:
            p[1][0].patches(xs, ys, fill_alpha=0, line_width=3, line_color=line_color)
            p[1][1].patches(xs, ys, fill_alpha=0, line_width=3, line_color=line_color)

    line_color='green'
    
    for region in regions_main:
        path_shapefile = '/home/jens/daten/geometries/geological_regions/BW/auswertungsregionen'
        shapefile = 'region_{}.shp'.format(region)

        gdf = gpd.read_file(regions_path + '/' + shapefile)

        if gdf.crs is None:
            gdf.set_crs(epsg=4326, inplace=True)  # WGS84 setzen

        # Reprojektion auf UTM Zone 32N (ETRS89)
        if gdf.crs.to_epsg() != 25832:
            gdf = gdf.to_crs(epsg=25832)
        # --- Shapefile-Geometrien extrahieren und umrechnen (auf km und mit Offset) ---
        xs = []
        ys = []
        for geom in gdf.geometry:
            if geom.geom_type == 'Polygon':
                x, y = geom.exterior.coords.xy
                xs.append([(xi - x_offset) / 1000 for xi in x])
                ys.append([(yi - y_offset) / 1000 for yi in y])
            elif geom.geom_type == 'MultiPolygon':
                for poly in geom.geoms:
                    x, y = poly.exterior.coords.xy
                    xs.append([(xi - x_offset) / 1000 for xi in x])
                    ys.append([(yi - y_offset) / 1000 for yi in y])

        p[0][0].patches(xs, ys, fill_alpha=0, line_width=2, line_color=line_color)
        p[0][1].patches(xs, ys, fill_alpha=0, line_width=2, line_color=line_color)
        if grid_drift is not None:
            p[1][0].patches(xs, ys, fill_alpha=0, line_width=2, line_color=line_color)
            p[1][1].patches(xs, ys, fill_alpha=0, line_width=2, line_color=line_color)

    grid = gridplot(p) 
    show(grid)

<h1> Data preparation </h1>

In [ ]:
bhs_val = df_filt[f"T_{T_selected}"].to_numpy()
bhs_coords = df_filt[['utm_easting','utm_northing']].to_numpy()
bhs_elev = df_filt["elevation"].to_numpy()

bhs_x, bhs_y = bhs_coords[:,0], bhs_coords[:,1]

bhs_val_cleaned = df_filt_cleaned[f"T_{T_selected}"].to_numpy()
bhs_coords_cleaned = df_filt_cleaned[['utm_easting','utm_northing']].to_numpy()
bhs_elev_cleaned = df_filt_cleaned["elevation"].to_numpy()

bhs_x_cleaned, bhs_y_cleaned = bhs_coords_cleaned[:,0], bhs_coords_cleaned[:,1]


<h1> Variogram </h1>

In [ ]:
class DoubleSpherical(gs.CovModel):
    def __init__(self, dim=2, var1=1.0, len1=10_000,
                 var2=2.0, len2=60_000, nugget=0.0):
        # wir definieren "var" nur als Summe für GSTools-Interna
        super().__init__(dim=dim, var=var1+var2, len_scale=max(len1, len2), nugget=nugget)
        self.var1, self.len1 = var1, len1
        self.var2, self.len2 = var2, len2
        self.nugget_val = nugget

    def variogram(self, r):
        """doppeltes sphärisches Variogramm"""
        def spherical(h, var, rng):
            h = np.asarray(h)
            gamma = np.where(
                h < rng,
                var * (1.5*(h/rng) - 0.5*(h/rng)**3),
                var
            )
            return gamma

        gamma1 = spherical(r, self.var1, self.len1)
        gamma2 = spherical(r, self.var2, self.len2)
        return self.nugget_val + gamma1 + gamma2

# --- Fit-Funktion für curve_fit ---
def double_spherical_func(r, var1, len1, var2, len2, nugget):
    model = DoubleSpherical(var1=var1, len1=len1,
                            var2=var2, len2=len2,
                            nugget=nugget)
    return model.variogram(r)

def spherical_func(r, var1, len1, nugget):
    model = DoubleSpherical(var1=var1, len1=len1,
                            var2=0., len2=0.,
                            nugget=nugget)
    return model.variogram(r)

    

def do_fit(func, bin_center, gamma, bin_counts, weighting=False):
    sigma = 1.0 / np.sqrt(bin_counts)

    if func == spherical_func:
        p0 = [2.0, 30_000, .0]
        bounds = ([0, 1000, 0], [10, 200_000, 2.])
        if weighting:
            popt, pcov = curve_fit(func, bin_center, gamma, p0=p0, bounds=bounds, sigma=sigma)
        else:
             popt, pcov = curve_fit(func, bin_center, gamma, p0=p0, bounds=bounds)           
        var1_fit, len1_fit, nugget_fit = popt
        var2_fit, len2_fit = 0., 0.
    elif func == double_spherical_func:
        p0 = [1.0, 10_000, 1.0, 20_000, 0.]
        bounds = ([0, 1000, 0, 0, 0], [10, 200_000, 10, 200_000, 5])
        if weighting:
            popt, pcov = curve_fit(func, bin_center, gamma, p0=p0, bounds=bounds, sigma=sigma)
        else:
            popt, pcov = curve_fit(func, bin_center, gamma, p0=p0, bounds=bounds)            
        var1_fit, len1_fit, var2_fit, len2_fit, nugget_fit = popt
    else:
        print("function not supported")
        return  0., 0., 0., 0., 0.

    return var1_fit, len1_fit, var2_fit, len2_fit, nugget_fit

def do_variogram_plot(fit_model, bin_center, gamma, bin_counts):
    
    gamma_pred = fit_model.variogram(bin_center)
    r2 = r2_score(gamma, gamma_pred)
    print(f"R² = {r2:.3f}")
    
    weights = bin_counts / np.sum(bin_counts)
    ss_res = np.sum(weights * (gamma - gamma_pred)**2)
    ss_tot = np.sum(weights * (gamma - np.average(gamma, weights=weights))**2)
    r2_weighted = 1 - ss_res/ss_tot
    print(f"R² weighted = {r2_weighted:.3f}")
    
    para = {}
    para['var1'] = fit_model.var1
    para['var2'] = fit_model.var2
    para['len_scale1'] = fit_model.len1
    para['len_scale2'] = fit_model.len2
    para['nugget'] = fit_model.nugget
    
    plot_variogram("Fitted DoubleSpherical", fit_model, bin_center, gamma, bin_counts, para, r2)


In [ ]:
# --- Empirisches Variogramm ---
bin_center, gamma, bin_counts = gs.vario_estimate(
    (bhs_x_cleaned, bhs_y_cleaned),
    bhs_val_cleaned,
    return_counts=True
)

var1_fit, len1_fit, var2_fit, len2_fit, nugget_fit = do_fit(
    spherical_func, 
    #double_spherical_func,
    bin_center, gamma, bin_counts, #weighting=True
)

print("Gefundene Parameter:")
print(f"var1={var1_fit:.3f}, len1={len1_fit:.0f}, var2={var2_fit:.3f}, len2={len2_fit:.0f}, nugget={nugget_fit:.3f}")

if dataset == 'RLP':
    nugget_fit = 1
    var1_fit = 2.1
    len1_fit = 25000
elif dataset == 'BW':
    len1_fit = 25000
    
print("Gesetzte Parameter:")
print(f"var1={var1_fit:.3f}, len1={len1_fit:.0f}, var2={var2_fit:.3f}, len2={len2_fit:.0f}, nugget={nugget_fit:.3f}")

# --- Modell mit Fit-Parametern ---
fit_model_global = DoubleSpherical(var1=var1_fit, len1=len1_fit,
                            var2=var2_fit, len2=len2_fit,
                            nugget=nugget_fit)

#do_variogram_plot(fit_model_global, bin_center, gamma, bin_counts)

In [ ]:

if logarithmic:
    residual = np.log(bhs_val_cleaned) - trend_global(df_filt_cleaned["elevation"], df_filt_cleaned[sealing_selected])
else:
    residual = bhs_val_cleaned - trend_global(df_filt_cleaned["elevation"], df_filt_cleaned[sealing_selected])

# --- Empirisches Variogramm ---
bin_center, gamma, bin_counts = gs.vario_estimate(
    (bhs_x_cleaned, bhs_y_cleaned),
    residual,
    return_counts=True
)

var1_fit, len1_fit, var2_fit, len2_fit, nugget_fit = do_fit(
    spherical_func, 
    #double_spherical_func,
    bin_center, gamma, bin_counts, #weighting=True
)

print("Gefundene Parameter:")
print(f"var1={var1_fit:.3f}, len1={len1_fit:.0f}, var2={var2_fit:.3f}, len2={len2_fit:.0f}, nugget={nugget_fit:.3f}")

if dataset == 'RLP':
    #nugget_fit = 0
    #var1_fit = 1.8
    len1_fit = 25000
elif dataset == 'BW':
    len1_fit = 25000

print("Gesetzte Parameter:")
print(f"var1={var1_fit:.3f}, len1={len1_fit:.0f}, var2={var2_fit:.3f}, len2={len2_fit:.0f}, nugget={nugget_fit:.3f}")

    
# --- Modell mit Fit-Parametern ---
fit_model_global_res = DoubleSpherical(var1=var1_fit, len1=len1_fit,
                            var2=var2_fit, len2=len2_fit,
                            nugget=nugget_fit)

#do_variogram_plot(fit_model_global_res, bin_center, gamma, bin_counts)

<h1> Kriging </h1>

<h2> Ordinary </h2>

In [ ]:
krig = gs.krige.Ordinary(model=fit_model_global, cond_pos=(bhs_x,bhs_y), cond_val=bhs_val, #unbiased=True
               )


srf, var = krig.structured([gridx, gridy])

do_bplot('Ordinary', dem_file, [0, 1000], [gridx, gridy],
         srf, var, df_filt, df_filt_cleaned, bhs_x, bhs_y, bhs_val, grid_drift=None)

<h2> Global residual </h2>

In [ ]:
if logarithmic:
    residual = np.log(bhs_val) - trend_global(df_filt["elevation"], df_filt[sealing_selected])
else:
    residual = bhs_val - trend_global(df_filt["elevation"], df_filt[sealing_selected])
    
# Residuen krigen (OK mit Mittelwert unbekannt—bei residuen oft ~0; SK mit mean=0 geht auch)
ok = gs.krige.Ordinary(
    model=fit_model_global_res,
    cond_pos=(bhs_x, bhs_y),
    cond_val=residual
)

# Residualfeld auf Grid
r_srf, r_var = ok.structured([gridx, gridy])

grid_drift = trend_global(grid_elevation_values, grid_sealing_values)

srf = r_srf + grid_drift


do_bplot('Global res', dem_file, [0, 1000], [gridx, gridy], srf, 
         r_var, df_filt, df_filt_cleaned, bhs_x, bhs_y, bhs_val, grid_drift=grid_drift, logarithmic=logarithmic)